In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import sys


# Create an empty model
m = gp.Model('Production Planning')

# Decision Variables
x1 = m.addVar(name="Month 1 Production")
y1 = m.addVar(name="Month 1 Extra Production")
x2 = m.addVar(name="Month 2 Production")
y2 = m.addVar(name="Month 2 Extra Production")
x3 = m.addVar(name="Month 3 Production")
y3 = m.addVar(name="Month 3 Extra Production")
x4 = m.addVar(name="Month 4 Production")
y4 = m.addVar(name="Month 4 Extra Production")


#################
## CONSTRAINTS ##
################

# Maximum storage capacity of warehouse is 100 tons
m.addConstr(x1 + y1 - 120 <= 100)
m.addConstr((x1+y1-120) + (x2+y2-160) <= 100)
m.addConstr((x1+y1-120) + (x2+y2-160) + (x3+y3-300) <= 100)

# xi must be less than or equal to pi
m.addConstr(x1 <= 140)
m.addConstr(x2 <= 150)
m.addConstr(x3 <= 140)
m.addConstr(x4 <= 160)

#yi must be less than or equal to qi
m.addConstr(y1 <= 50)
m.addConstr(y2 <= 75)
m.addConstr(y3 <= 70)
m.addConstr(y4 <= 80)

#constraints for meeting minumum demand
m.addConstr(x1+y1 >= 120)
m.addConstr((x1+y1+x2+y2) - 120 >= 160)
m.addConstr((x1+y1+x2+y2+x3+y3) - (120+160) >= 300)
m.addConstr((x1+y1+x2+y2+x3+y3+x4+y4) - (120+160+300) == 200)

#warehouse must be empty at end of month 4
m.addConstr((x1+y1+x2+y2+x3+y3+x4+y4) - (120+160+300+200) == 0)

#The regular (non-extra) production in each month must be at least 10% of the total production of the 
#first three months 
m.addConstr(x1 >= .1*(x1+y1+x2+y2+x3+y3))
m.addConstr(x2 >= .1*(x1+y1+x2+y2+x3+y3))
m.addConstr(x3 >= .1*(x1+y1+x2+y2+x3+y3))
m.addConstr(x4 >= .1*(x1+y1+x2+y2+x3+y3))


# Objective is to minimize the total cost
obj = x1*440+y1*(440+260)+x2*440+y2*(440+260)+x3*440+y3*(440+260)+x4*440+y4*(440+260)+5*(x1+y1-120) + 5*(x1+y1+x2+y2-160-120) + 5*(x1+y1+x2+y2+x3+y3-160-120-300)
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()


#Print optimal value of the objective function
print('\nValue of objective function: %g' % m.ObjVal)

# Print optimal values for the decision variables and cost savings
print('\nDecision variables:')
tmp=0
#cost_reduction=0
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 20 rows, 8 columns and 73 nonzeros
Model fingerprint: 0xc62f8cc0
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [4e+02, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 8e+02]
Presolve removed 20 rows and 8 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9317500e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.06 seconds (0.00 work units)
Optimal objective  3.931750000e+05

Value of objective function: 393175

Decision variables:
Month 1 Production = 140
Month 1 Extra Production = 5
Month 2 Production = 150
Month 2 Extra Production = 75
Mon